# HESA 

Some code to collect HESA data and merge it with university metadata in order to create indicators about skills supply in the UK.

We are interested in the following indicators:

* Research staff: https://www.hesa.ac.uk/data-and-analysis/staff/working-in-he
* Research spaces: https://www.hesa.ac.uk/data-and-analysis/estates/table-1
* STEM graduates produced: https://www.hesa.ac.uk/data-and-analysis/students/what-study#
* PhD graduates produced: this is also in https://www.hesa.ac.uk/data-and-analysis/students/what-study#


See [this table](https://docs.google.com/spreadsheets/d/1V2fAQcvuLsoImwo6uLdyIK3x80pBNoX97CxsxkjvRP4/edit?usp=sharing) for more information.


## Preamble

In [ ]:
%run ../notebook_preamble.ipy

In [ ]:
import csv
import zipfile
import io

In [ ]:
import seaborn as sn
from nuts_finder import NutsFinder

### Functions

#### Simple utilities

In [ ]:
def tidy_cols(my_csv):
    '''
    Tidies column names ie lower and replace spaces with underscores
    
    '''
    
    return([re.sub(' ','_',col.lower()) for col in my_csv.columns])

In [ ]:
def filter_data(data,var_val_pairs):
    '''
    We use this to filter the data more easily than using pandas subsetting
    
    Args:
        data (df) is a dataframe
        var_val pairs (dict) is a dictionary where the keys are variables and the value are values

    
    '''
    d = data.copy()
    
    for k,v in var_val_pairs.items():
        d = d.loc[d[k]==v]
        
    return(d.reset_index(drop=True))
    

#### Data collection

In [ ]:
def hesa_parser(url,out_name,skip=16,encoding='utf-8'):
    '''
    Function to obtain and parse data from the HESA website 
    
    Args:
        url (str) is the location of the csv file
        out_name (str) is the saved name of the file
        skip is the number of rows to skip (we could automate this by looking for rows at the top with lots of nans)
    
    '''
    
    #Request and parse
    rs = requests.get(url)
    
    #Parse the file
    parsed = rs.content.decode(encoding)
    
    #Save it
    
    with open(f'../../data/raw/hesa/{out_name}.txt','w') as outfile:
        outfile.write(parsed)
        
    #Read it.
    my_csv = pd.read_csv(f'../../data/raw/hesa/{out_name}.txt',skiprows=skip)
    
    #Clean column names
    my_csv.columns = tidy_cols(my_csv)
    
    
    return(my_csv)

    
    

#### Data processing

In [ ]:
def gimme_nuts(lat,lon,level=2):
    '''
    Function to extract nuts information from a pair of coordinates
    
    Args:
        lat (float) is the latitude
        lon (float) is the longitude
        level (int) is the NUTS level we want
        
    
    '''
    
    info = nf.find(lat=round(lat,5),lon=round(lon,5))
    try:
        nuts_id = [x['NUTS_ID'] for x in info if x['LEVL_CODE']==level][0]
        nuts_name = [x['NUTS_NAME'] for x in info if x['LEVL_CODE']==level][0]
    #print(info)
    
    #nuts_id = info[level]['NUTS_ID']
    #nuts_name = info[level]['NUTS_NAME']
    
    except:
        print(f'failed with {np.round(lat,5)},{np.round(lon,5)}')
        nuts_id = np.nan
        nuts_name=np.nan
    
    return([nuts_id,nuts_name])

In [ ]:
def compare_data(df_1,df_2,id_1,id_2,name_1,name_2):
    '''
    We use this function to check if the ids in two datasets we are merging are consistent.
    
    Args:
        dfs are the dfs we want to compare
        ids are the ids we want to check
        names are the names we want to use to explore the data
    
    '''
    
    print('In 1 but not in 2')
    print('==================')
    d1_miss = set(df_1[id_1].dropna())-set(df_2[id_2])
    print(set(df_1.loc[[x in d1_miss for x in df_1[id_1]]][name_1]))
    
    print('\n')

    
    print('In 2 but not in 1')
    print('==================')
    d2_miss = set(df_2[id_2].dropna())-set(df_1[id_1])
    print(set(df_2.loc[[x in d2_miss for x in df_2[id_2]]][name_2]))

    

#### Create NUTS aggregations

In [ ]:
def make_nuts_estimate(data,nuts_lookup,counter,name):
    '''
    This function takes hesa data and creates a nuts estimate
    
    Args:
        data (df) where we have already selected variables of interest eg mode of employment
        nuts (dict) is the ukprn - nuts name and code lookup
        counter (str) is the variable with counts that we are interested in
    
    '''
    
    d = data.copy()
    
    #Add the nuts names and codes
    d['nuts_name'],d['nuts_code'] = [[nuts_lookup[ukprn][var] if ukprn in nuts_lookup.keys() else np.nan for ukprn in data['ukprn']] for
                                     var in ['nuts_name','nuts_code']]
    
    #We are focusing on numbers
    d[counter] = d[counter].astype(float)
    
    out = d.groupby(['nuts_name','nuts_code'])[counter].sum()
    
    out.name = name
    
    return(out)

In [ ]:
def multiple_nuts_estimates(data,nuts_lookup,variables,select_var,value):
    '''
    Creates NUTS estimates for multiple variables.
    
    Args:
        data (df) is the filtered dataframe
        select_var (str) is the variable we want to use to select values
        nuts_lookup (dict) is the lookup between universities and nuts
        variables (list) is the list of variables for which we want to generate the analysis
        value (str) is the field that contains the numerical value we want to aggregate in the dataframe
    
    '''
    
    concat = pd.concat([make_nuts_estimate(data.loc[data[select_var]==m],nuts_lookup,value,m) for m in 
              variables],axis=1)
    
    return(concat)
        
    

#### Directories etc

In [ ]:
# Create a hesa directory in raw and processed

In [ ]:
if 'hesa' not in os.listdir('../../data/raw'):
    os.mkdir('../../data/raw/hesa')
    
if 'hesa' not in os.listdir('../../data/processed'):
    os.mkdir('../../data/processed/hesa')

## Collect data

### University metadata

The [learning providers website](http://learning-provider.data.ac.uk/) contains information about universities.



In [ ]:
uni_meta = pd.read_csv('http://learning-provider.data.ac.uk/data/learning-providers-plus.csv')

In [ ]:
uni_meta.columns = tidy_cols(uni_meta)

In [ ]:
uni_meta.head()

Label universities

In [ ]:
#This initialises an object to label lons and lats with their NUTS code
nf = NutsFinder()

In [ ]:
#Create a dict from ukprn to name, nuts2
uni_nuts = {row['ukprn']:{'name':row['view_name'],
                          'nuts_code':gimme_nuts(lat=row['latitude'],lon=row['longitude'])[0],
                          'nuts_name':gimme_nuts(lat=row['latitude'],lon=row['longitude'])[1]} for rid,row in uni_meta.iterrows()}

#### Research staff

In [ ]:
res_staff = hesa_parser('https://www.hesa.ac.uk/data-and-analysis/staff/table-1.csv','staff',skip=24)

We also downloaded staff qualifications but probably won't use it

In [ ]:
qual_staff = hesa_parser('https://www.hesa.ac.uk/data-and-analysis/staff/table-8.csv','qual_staff')

### Research spaces

In [ ]:
spaces = hesa_parser('https://www.hesa.ac.uk/data-and-analysis/estates/data.csv','spaces',11)

### Stem graduates

This is a larger zip file so we have to use a different approach

In [ ]:
#Request
rs = requests.get('https://www.hesa.ac.uk/data-and-analysis/students/table-13.csv')

In [ ]:
#Unzip and save the file
out = zipfile.ZipFile(io.BytesIO(rs.content)).extract('table-13.csv','../../data/raw/hesa/')

In [ ]:
graduates = pd.read_csv(out,skiprows=17)

In [ ]:
graduates.columns = tidy_cols(graduates)

In [ ]:
graduates.head()

## 2. Processing

Processing involves:

1. Select variables we want to use in the analysis (eg year, mode of study)
2. Label the data with the NUTS information
3. Group over NUTS and generate estimate

We can probably create a function to do 2 and 3 taking the subset data as input


### Do all the universities in HESA have metadata?

In [ ]:
compare_data(res_staff,uni_meta,'ukprn','ukprn','he_provider','view_name')

In [ ]:
compare_data(spaces,uni_meta,'ukprn','ukprn','he_provider','view_name')

In [ ]:
# Graduates takes too long to run!

#We create a shorter version

In [ ]:
grad_short = graduates.loc[(graduates['level_of_study']=='All')&((graduates['mode_of_study']=='Full-time'))]

In [ ]:
compare_data(grad_short,uni_meta,'ukprn','ukprn','he_provider','view_name')

These are small universities - I have checked names and found that the difference between sets isn't caused by mismatches in codes (eg the same university having different codes in different sources)

### Make indicators

#### 1. Number of research staff

In [ ]:
res_staff.head()

In [ ]:
from itertools import combinations

In [ ]:
def check_categories(data,columns):
    '''
    This counts frequencies of categorical variables. We use it to decide what variables to choose, and to avoid double counting
    
    Args:
        Data (df) is the data
        Columns (list) are the categorical variables we want to check
    
    '''
    print('FREQUENCIES')
    print('===========')
    
    print('\n')
    #We check frequencies
    
    for var in columns:
    
        print(var)
        print('=====')
        print(data[var].value_counts())

        print('\n')
        
    print('CROSSTABS')
    print('===========')
    
    #We check combinations
    
    combs = list(combinations(columns,2))
    
    for comb in combs:
        print(comb[0]+' x '+comb[1])
        print('=====')
        print(pd.crosstab(data[comb[0]],data[comb[1]]))
        
        print('\n')
        
        
    


In [ ]:
#We check categories in interesting columns
interesting_cols = ['mode_of_employment','atypical_marker','contract_marker','academic_year','activity_standard_occupational_classification']

#check_categories(res_staff,interesting_cols)

In [ ]:
res_staff_filter = {'mode_of_employment':'All','contract_marker':'Academic','academic_year':'2017/18',
                   'activity_standard_occupational_classification':'Total academic staff',
                   'country_of_he_provider':'All','region_of_he_provider':'All'}

In [ ]:
res_filtered = filter_data(res_staff,res_staff_filter)

len(res_filtered)

In [ ]:
nuts_academics = make_nuts_estimate(res_filtered,uni_nuts,'number','academic_staff')

In [ ]:
nuts_academics.sort_values(ascending=False)

#### 2. Research space

See some variable definitions for estates [here](https://www.hesa.ac.uk/support/definitions/estates)

In [ ]:
spaces.head()

In [ ]:
sp_interesting_cols = ['academic_year','country_of_he_provider','region_of_he_provider','category_marker','table']

#check_categories(spaces,sp_interesting_cols)

This contains a lot of information. We will only focus on a couple of variables:

* Total number of buildings
* Total site area
* Research income
* Research student FTE
* Total site area (hectares)

In [ ]:
#We focus on the most recent year
space_filter = {'academic_year':'2017/18'}

space_filtered = filter_data(spaces,space_filter)

In [ ]:
space_vars = ['Research income (£)','Research student FTE','Total number of buildings','Total site area (hectares)']

nuts_spaces = multiple_nuts_estimates(space_filtered,uni_nuts,space_vars,'category_marker','value')

nuts_spaces.head()

### 3. Number of STEM graduates

The graduates file is quite big so I focus on grad short, which considers all full time graduates

In [ ]:
grad_short.head()

In [ ]:
grad_interesting_columns = ['level_of_study','mode_of_study','country_of_he_provider',
                            'region_of_he_provider','subject_of_study_marker','subject_of_study']

#check_categories(grad_short,grad_interesting_columns)

In [ ]:
grad_filter = {'country_of_he_provider':'All','region_of_he_provider':'All','academic_year':'2017/18'}

grad_filtered = filter_data(grad_short,grad_filter)

In [ ]:
disciplines = set(grad_filtered['subject_of_study'])

nuts_disciplines = multiple_nuts_estimates(grad_filtered,uni_nuts,disciplines,'subject_of_study','number')

In [ ]:
nuts_disciplines.head()

### Number of postgraduates

This is a flavour of the variable above where we count the number of research postgraduates

In [ ]:
#We will filter the data to focus on full time postgraduate researchers
post_grad_filter = grad_filter.copy()

post_grad_filter['level_of_study'] = 'Postgraduate (research)'
post_grad_filter['mode_of_study'] = 'Full-time'

In [ ]:
post_grad_filtered = filter_data(graduates,post_grad_filter)

In [ ]:
nuts_postgrads = make_nuts_estimate(post_grad_filtered,uni_nuts,'number','postgrad_research')

In [ ]:
nuts_postgrads.head(n=10)

### Conclusion

We ended with the following datasets for academic year 2017/18:

* `nuts_academics` contains academic staff per NUTS2
* `nuts_spaces` contains key scale variables per NUTS2. This includes:
  * Research income ([Definition](https://www.hesa.ac.uk/collection/c17042/a/firei))
  * Research student FTE ([definition](https://www.hesa.ac.uk/collection/c17042/a/stftere) - these are doctorates or masters)
  * Total number of buildings
  * Total site area ([definition](https://www.hesa.ac.uk/collection/c17042/a/sitearea))
* `nuts_disciplines` contains student enrolment by discipline
* `nuts_postgrads` contains postgraduates in research per discipline (Full time) This should be strongly correlated (if not the same) as Research student FTE in `nuts_spaces`



In [ ]:
nuts_disciplines[['(N7) Office skills','(A0) Broadly-based programmes within medicine and dentistry']].head()

There are a couple of disciplines with zero levels of activity

In [ ]:
all_vars = pd.concat([nuts_academics,nuts_spaces,nuts_disciplines,nuts_postgrads],axis=1)

all_vars.drop(axis=1,labels=['(N7) Office skills','(A0) Broadly-based programmes within medicine and dentistry'],inplace=True)

all_vars.columns = tidy_cols(all_vars)

Could it be that they are never studied part time

In [ ]:
#Save
all_vars.to_csv(f'../../data/processed/hesa/{today_str}_hesa_data_nuts_2.csv')

#### Cluster map

In [ ]:
sn.clustermap(all_vars.corr())